In [1]:
import warnings
warnings.simplefilter('ignore')

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fast_ml.feature_selection import get_constant_features

from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import PowerTransformer, StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pickle
import os
from sklearn.ensemble import RandomForestRegressor
import lightgbm
import statsmodels.api as sm

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import keras_tuner as kt
from keras.regularizers import l2

from tensorflow.keras.models import load_model

%matplotlib inline

In [3]:
test = pd.read_csv('C:/Users/rsk13/Desktop/CLTV/data_source/test_koRSKBP.csv')
test.shape

(59595, 11)

In [4]:
test

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy
0,89393,Female,Rural,High School,5L-10L,0,6,2134,More than 1,B,Silver
1,89394,Female,Urban,High School,2L-5L,0,4,4102,More than 1,A,Platinum
2,89395,Male,Rural,High School,5L-10L,1,7,2925,More than 1,B,Gold
3,89396,Female,Rural,Bachelor,More than 10L,1,2,0,More than 1,B,Silver
4,89397,Female,Urban,High School,2L-5L,0,5,14059,More than 1,B,Silver
...,...,...,...,...,...,...,...,...,...,...,...
59590,148983,Male,Rural,Bachelor,5L-10L,1,6,0,1,A,Silver
59591,148984,Male,Urban,High School,5L-10L,0,6,0,1,C,Gold
59592,148985,Female,Urban,High School,2L-5L,1,4,3323,More than 1,B,Platinum
59593,148986,Male,Rural,Bachelor,5L-10L,0,1,2886,More than 1,A,Silver


### Data Preprocessing 

##### similar to train

In [5]:
test.drop(['id'], axis = 1, inplace = True)

In [6]:
test.head()

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy
0,Female,Rural,High School,5L-10L,0,6,2134,More than 1,B,Silver
1,Female,Urban,High School,2L-5L,0,4,4102,More than 1,A,Platinum
2,Male,Rural,High School,5L-10L,1,7,2925,More than 1,B,Gold
3,Female,Rural,Bachelor,More than 10L,1,2,0,More than 1,B,Silver
4,Female,Urban,High School,2L-5L,0,5,14059,More than 1,B,Silver


In [7]:
test['income'] = test['income'].map({'<=2L': 1, '2L-5L': 2, '5L-10L': 3, 'More than 10L': 4})

### 

test['num_policies'] = test['num_policies'].map({'1': 1, 'More than 1':2})

### 

gender = pd.get_dummies(test['gender'], drop_first = True).astype(int)
test = pd.concat([test, gender], axis = 1)

### 

area = pd.get_dummies(test['area'], drop_first = True).astype(int)
test = pd.concat([test, area], axis = 1)

###

qualifications = pd.get_dummies(test['qualification'], drop_first = True).astype(int)
test = pd.concat([test, qualifications], axis = 1)

### 

policy = pd.get_dummies(test['policy'], drop_first = True).astype(int)

test = pd.concat([test, policy], axis = 1)

###

test['type_of_policy'] = test['type_of_policy'].map({'Silver' : 1, 'Gold' : 2, 'Platinum' : 3})

###
test.drop(columns = ['gender', 'area', 'qualification', 'policy'], axis = 1, inplace = True)

In [8]:
test

,income,marital_status,vintage,claim_amount,num_policies,type_of_policy,Male,Urban,High School,Others,B,C
0,3,0,6,2134,2,1,0,0,1,0,1,0
1,2,0,4,4102,2,3,0,1,1,0,0,0
2,3,1,7,2925,2,2,1,0,1,0,1,0
3,4,1,2,0,2,1,0,0,0,0,1,0
4,2,0,5,14059,2,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59590,3,1,6,0,1,1,1,0,0,0,0,0
59591,3,0,6,0,1,2,1,1,1,0,0,1
59592,2,1,4,3323,2,3,0,1,1,0,1,0
59593,3,0,1,2886,2,1,1,0,0,0,0,0


In [24]:
# Load the model
model_dnn = load_model('C:/Users/rsk13/Desktop/CLTV_New/model_weight/dnn.h5')

model_linreg = pickle.load(open('C:/Users/rsk13/Desktop/CLTV_New/model_weight/lin_reg.pkl', 'rb'))

model_rf = pickle.load(open('C:/Users/rsk13/Desktop/CLTV_New/model_weight/tuned_rf.pkl', 'rb'))

In [18]:
y_pred_dnn = model_dnn.predict(test)

1863/1863 [==============================] - 4s 2ms/step


In [19]:
y_pred_linreg = model_linreg.predict(test)

In [25]:
y_pred_rf = model_rf.predict(test)

In [26]:
y_pred_dnn

array([[-2078879.8],
       [-4086333.5],
       [-2886259.2],
       ...,
       [-3291680.8],
       [-2844392.2],
       [-5969579.5]], dtype=float32)

In [27]:
y_pred_linreg

array([18958068.65300976, 36311624.6125314 , 25927691.28723043, ...,
       29438676.19131418, 25588972.97750067, 52491830.8687402 ])

In [28]:
y_pred_rf

array([132031.74245135, 150074.87205636, 132031.74245135, ...,
       132031.74245135, 149891.10637964, 132031.74245135])

### The values from Deep Net and Linear Regression are really off. So will select the next best thing, Random Forest.

In [30]:
test_final = pd.read_csv('C:/Users/rsk13/Desktop/CLTV/data_source/test_koRSKBP.csv')

In [32]:
id = test_final.iloc[:,0]
CLTV = y_pred_rf.tolist()

submission = pd.DataFrame(list(zip(id, CLTV)), columns = ['id', 'CLTV'])

submission

,id,CLTV
0,89393,132031.742451
1,89394,150074.872056
2,89395,132031.742451
3,89396,104953.191920
4,89397,132031.742451
...,...,...
59590,148983,115673.290074
59591,148984,115673.290074
59592,148985,132031.742451
59593,148986,149891.106380
